# Bushfire Detection Model Development
Using the Edge Impulse Python SDK with TensorFlow and Keras to train model to detect bushfires vs. no bushfire within a given image.

#Install Dependencies

In [ ]:
!python -m pip install tensorflow==2.12.0 edgeimpulse

# Import Packages

In [ ]:
from tensorflow import keras
import edgeimpulse as ei
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import os

# Setup

In [ ]:
# Settings
ei.API_KEY = "ei_c90f6349e4f2624274eff30ee840a3e6412c3135d1651ee15515404d290ce842"
labels = ['fire', 'nofire']
num_classes = len(labels)
deploy_filename = "bushfire_detection_model_cpp.zip"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Prepare Dataset
Organise files between 'Training and Validation' and 'Testing' folders containing 'fire' and 'nofire' images.


In [ ]:
# Define paths for your dataset
train_val_directory = 'drive/MyDrive/forest_fire/Training and Validation'
test_directory = 'drive/MyDrive/forest_fire/Testing'

# Create an ImageDataGenerator for training and validation
train_val_datagen = ImageDataGenerator(
    rescale=1.0 / 255,        # Normalize pixel values
    validation_split=0.2      # Use 20% of the training data for validation
)

# Create generators for training and validation data
train_generator = train_val_datagen.flow_from_directory(
    train_val_directory,
    target_size=(128, 128),   # Resize images to 128x128
    batch_size=32,             # Number of images to return in each batch
    class_mode='categorical',       # Binary classification (fire or no fire)
    subset='training',          # Set as training data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

validation_generator = train_val_datagen.flow_from_directory(
    train_val_directory,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',       # Binary classification
    subset='validation',        # Set as validation data
    classes=['fire','nofire'] # Explicitly specify the class names
)

Setup of 'Testing' directory

In [ ]:
# Create an ImageDataGenerator for testing (no split needed)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Create a generator for the test data
test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',       # Binary classification
    shuffle=False,             # Don't shuffle test data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

#Building and compiling model

In [ ]:
# Build your model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    # tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.GlobalAveragePooling2D(),
    # tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Testing model setup

In [ ]:
print(train_generator.class_indices)

# Training Model

In [ ]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

#Test Accuracy and Evaluation

In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Accuracy: {test_accuracy:.2f}')

# Model to System Compatibility

In [ ]:
# List the available profile target devices
ei.model.list_profile_devices()

In [ ]:
#Estimate the RAM, ROM, and inference time for our model on the target hardware family
try:
    profile = ei.model.profile(model=model,
                               device='openmv-h7p')
    print(profile.summary())
except Exception as e:
    print(f"Could not profile: {e}")

# Deploying and Downloading Model

In [ ]:
import tensorflow as tf

# Define the output and input types for your model
model_output_type = ei.model.output_type.Classification(labels=labels)
model_input_type = ei.model.input_type.ImageInput()

# Define filenames
saved_model_dir = "saved_model"
tflite_model_filename = "model.tflite"
deploy_filename = "tflite_model_zip.zip"

# Save your model as a TensorFlow SavedModel
model.save(saved_model_dir)  # Save the model correctly

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the converted TFLite model
with open(tflite_model_filename, "wb") as f:
    f.write(tflite_model)

print(f"Model converted to TensorFlow Lite and saved as {tflite_model_filename}")

# Deploy the TFLite model as a .zip file
import zipfile

try:
    with zipfile.ZipFile(deploy_filename, 'w') as zipf:
        zipf.write(tflite_model_filename)
    print(f"Model successfully packaged as {deploy_filename}")
except Exception as e:
    print(f"Could not create zip file: {e}")


In [ ]:
from google.colab import files
import shutil

shutil.make_archive("saved_model", 'zip', "saved_model")
files.download("saved_model.zip")

# Model Specifications
Code to check if model has been trained correctly

In [ ]:
model.summary()